# Balance de energía

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate
import import_ipynb

In [2]:
from calculos_Q_invierno import Q_oficinas, dot_Q_oficinas, T_oficinas, T_exterior_inv
Q_oficinas_invierno = Q_oficinas
dot_Q_oficinas_invierno = dot_Q_oficinas
T_oficinas_invierno = T_oficinas
T_exterior_invierno = T_exterior_inv

from calculos_Q_verano import Q_oficinas, dot_Q_oficinas, T_oficinas, T_exterior_ver
Q_oficinas_verano = Q_oficinas
dot_Q_oficinas_verano = dot_Q_oficinas
T_oficinas_verano = T_oficinas
T_exterior_verano = T_exterior_ver

Coeficiente global de transferencia de calor tabique tipo 1: 0.62 W/Km^2
Coeficiente global de transferencia de calor tabique tipo 2: 0.62 W/Km^2
Coeficiente global de transferencia de calor tabique tipo 3: 0.62 W/Km^2
Coeficiente global de transferencia de calor tabique tipo 4 por unidad de ancho: 0.57 W/Km
Coeficiente global de transferencia de calor tabique tipo 5: 0.66 W/Km^2
Coeficiente global de transferencia de calor muro hormigón armado: 8.15 W/Km^2
Coeficiente global de transferencia de calor en el suelo: 6.52 W/Km^2
Conductividad térmica aire quieto a 25 C: 0.0258 W/mK
Coeficiente global de transferencia de calor ventana tipo 1: 2.10 W/Km^2
Coeficiente global de transferencia de calor ventana MV17: 200.00 W/Km^2
Coeficiente global de transferencia de calor ventana MV18: 200.00 W/Km^2
Coeficiente global de transferencia de calor ventana MV19: 200.00 W/Km^2
Coeficiente global de transferencia de calor puerta tipo PM05: 2.37 W/Km^2
Coeficiente global de transferencia de calor im

Humedad:
Media humedad relativa:
- Diciembre 23: 51.5
- Enero 24: 50.1
- Febrero 24: 53.5
- Marzo 24: 55.6
--------
- Junio 24: 84.9
- Julio 24: 76.9
- Agosto 24: 78.1
- Septiembre 24: 67.4

In [3]:
humedad_verano = 52.675
humedad_invierno = 78.825

**Balance de masa**

$$
\dot{m}W_{exterior} + \dot{m}_{gen} = \dot{m}W_{interior}
$$

**Balance de energía**

$$
\dot{Q}_{in} + \dot{Q}_{gen} + \dot{m}h_{exterior} = \dot{Q}_{loss} + \dot{m}h_{interior}
$$

In [4]:
# Cálculo de Q_gen
mets = 1.5 * 58.2
n_personas = 1.5
peso = 84.9
altura = 1.75
area_superficial = 0.202 * peso**0.425 * altura**0.725
Q_gen_personas = n_personas * mets * area_superficial
print(f"Energía generada por las personas en la oficina: {Q_gen_personas:.2f} W")

Energía generada por las personas en la oficina: 262.09 W


Fórmulas de psicrometría

In [5]:
def humedad_absoluta(T, RH):
    """Calcula la humedad absoluta (kg/m3) a partir de la temperatura (°C) y la humedad relativa (%)."""
    # Presión de saturación del agua (Pa)
    P_sat = 610.78 * np.exp((17.27 * T) / (T + 237.3))
    # Presión parcial del vapor de agua (Pa)
    P_vapor = RH / 100 * P_sat
    # Humedad absoluta (kg/m3)
    humedad_abs = 0.622 * P_vapor / (101325 - P_vapor)   # Convertir a g/m3
    return humedad_abs

def entalpia_aire(T, RH):
    """Calcula la entalpía del aire (kJ/kg) a partir de la temperatura (°C) y la humedad relativa (%)."""
    humedad_abs = humedad_absoluta(T, RH) / 1000  # Convertir a kg/m3
    h = 1.006 * T + humedad_abs * (2501 + 1.86 * T)  # kJ/kg
    return h*1e3

m_gen = n_personas * 50/(1000*3600)  # kg/s, asumiendo 0.1 kg/s por persona CAMBIAR

## Cálculos para el invierno

Resolver sistema de ecuaciones para determinar $\dot{Q}_{in}$ y $\dot{m}$

In [6]:
W_interior = humedad_absoluta(T_oficinas, 50)  # kJ/kg, asumiendo 50% RH interior
W_exterior_inv = humedad_absoluta(T_exterior_inv, humedad_invierno)  # kJ/kg, asumiendo 80% RH exterior invierno

m = m_gen / (-W_exterior_inv + W_interior)  # kg/s
m = 2.14 * 4.14*3.78*2.71 * 1.1 /3600 # kg/s
print(f"Caudal de aire necesario para remover la humedad: {m:.5f} kg/s")

h_exterior_inv = entalpia_aire(T_exterior_inv, humedad_invierno)  # kJ/kg
h_interior = entalpia_aire(T_oficinas, 50)  # kJ/kg

total = 0
for oficina, perdidas in dot_Q_oficinas_invierno.items():
    Q_in = perdidas + m * (h_interior - h_exterior_inv) - Q_gen_personas  # W
    total += Q_in


    #print(f'Perdidas --> {perdidas}\nHinterior --> {h_interior}\nHexterior --> {h_exterior_inv}\nm --> {m}\nQgen --> {Q_gen_personas}')
    print(f"Potencia necesaria para calefacción en {oficina} durante el invierno: {Q_in*1e-3:.2f} kW")

print(f"Potencia total necesaria para calefacción durante el invierno: {total*1e-3:.2f} kW")

Caudal de aire necesario para remover la humedad: 0.02773 kg/s
Potencia necesaria para calefacción en Oficina 1 durante el invierno: 1.48 kW
Potencia necesaria para calefacción en Oficina 2 durante el invierno: 1.62 kW
Potencia necesaria para calefacción en Oficina 3 durante el invierno: 1.62 kW
Potencia necesaria para calefacción en Oficina 4 durante el invierno: 1.73 kW
Potencia necesaria para calefacción en Oficina 5 durante el invierno: 1.73 kW
Potencia necesaria para calefacción en Oficina 6 durante el invierno: 1.62 kW
Potencia necesaria para calefacción en Oficina 7 durante el invierno: 1.62 kW
Potencia necesaria para calefacción en Oficina 8 durante el invierno: 1.86 kW
Potencia necesaria para calefacción en Oficina 9 durante el invierno: 1.87 kW
Potencia necesaria para calefacción en Oficina 10 durante el invierno: 1.59 kW
Potencia necesaria para calefacción en Oficina 11 durante el invierno: 1.59 kW
Potencia necesaria para calefacción en Oficina 12 durante el invierno: 1.54 k

## Energía Invierno

In [7]:
Q_persona_invierno = Q_gen_personas * 10 * 22
Q_aire_dif = m * (h_interior - h_exterior_inv) * 10 * 22

total = 0
for oficina, perdidas in Q_oficinas_invierno.items():
    #print(perdidas)
    Q_in = perdidas*22 + Q_aire_dif - Q_persona_invierno  # Wh
    total += Q_in
    #print(f'Perdidas --> {perdidas}\nHinterior --> {h_interior}\nHexterior --> {h_exterior_inv}\nm --> {m}\nQgen --> {Q_gen_personas}')
    print(f"Potencia necesaria para calefacción en {oficina} durante un mes de invierno: {(Q_in*1e-3):.2f} kWh")
print(f"Potencia total necesaria para calefacción en todas las oficinas durante un mes de invierno: {(total*1e-3):.2f} kWh")

Potencia necesaria para calefacción en Oficina 1 durante un mes de invierno: 274.62 kWh
Potencia necesaria para calefacción en Oficina 2 durante un mes de invierno: 313.32 kWh
Potencia necesaria para calefacción en Oficina 3 durante un mes de invierno: 297.10 kWh
Potencia necesaria para calefacción en Oficina 4 durante un mes de invierno: 314.23 kWh
Potencia necesaria para calefacción en Oficina 5 durante un mes de invierno: 314.23 kWh
Potencia necesaria para calefacción en Oficina 6 durante un mes de invierno: 313.59 kWh
Potencia necesaria para calefacción en Oficina 7 durante un mes de invierno: 313.59 kWh
Potencia necesaria para calefacción en Oficina 8 durante un mes de invierno: 359.93 kWh
Potencia necesaria para calefacción en Oficina 9 durante un mes de invierno: 368.77 kWh
Potencia necesaria para calefacción en Oficina 10 durante un mes de invierno: 310.42 kWh
Potencia necesaria para calefacción en Oficina 11 durante un mes de invierno: 310.42 kWh
Potencia necesaria para calefa

## Cálculos para el verano

In [8]:
W_interior = humedad_absoluta(T_oficinas, 50)  # kJ/kg, asumiendo 50% RH interior
W_exterior_ver = humedad_absoluta(T_exterior_ver, humedad_verano)  # kJ/kg, asumiendo 80% RH exterior invierno

m = m_gen / (W_exterior_ver - W_interior)  # kg/s
print(f"Caudal de aire necesario para remover la humedad: {m:.2f} kg/s")

h_exterior_ver = entalpia_aire(T_exterior_ver, humedad_verano)  # kJ/kg
h_interior = entalpia_aire(T_oficinas, 50)  # kJ/kg

total = 0
for oficina, perdidas in dot_Q_oficinas_verano.items():
    Q_in = perdidas + m * (h_interior - h_exterior_ver) - Q_gen_personas  # W
    total += Q_in
    print(f"Potencia necesaria para climatización en {oficina} durante el verano: {Q_in*1e-3:.2f} kW")

print(f"Potencia total necesaria para climatización durante el verano: {total*1e-3:.2f} kW")

Caudal de aire necesario para remover la humedad: 0.00 kg/s
Potencia necesaria para climatización en Oficina 1 durante el verano: 0.60 kW
Potencia necesaria para climatización en Oficina 2 durante el verano: 0.68 kW
Potencia necesaria para climatización en Oficina 3 durante el verano: 0.68 kW
Potencia necesaria para climatización en Oficina 4 durante el verano: 0.75 kW
Potencia necesaria para climatización en Oficina 5 durante el verano: 0.75 kW
Potencia necesaria para climatización en Oficina 6 durante el verano: 0.69 kW
Potencia necesaria para climatización en Oficina 7 durante el verano: 0.69 kW
Potencia necesaria para climatización en Oficina 8 durante el verano: 0.83 kW
Potencia necesaria para climatización en Oficina 9 durante el verano: 0.84 kW
Potencia necesaria para climatización en Oficina 10 durante el verano: 0.67 kW
Potencia necesaria para climatización en Oficina 11 durante el verano: 0.67 kW
Potencia necesaria para climatización en Oficina 12 durante el verano: 0.64 kW
P

## Energía Verano

In [12]:
Q_persona_invierno = Q_gen_personas * 10 * 22
Q_aire_dif =  m * (h_interior - h_exterior_ver) * 10 * 22

total = 0
for oficina, perdidas in Q_oficinas_verano.items():
    #print(perdidas)
    Q_in = perdidas*22 + Q_aire_dif - Q_persona_invierno  # Wh
    total += Q_in
    #print(f'Perdidas --> {perdidas}\nHinterior --> {h_interior}\nHexterior --> {h_exterior_inv}\nm --> {m}\nQgen --> {Q_gen_personas}')
    print(f"Potencia necesaria para calefacción en {oficina} durante un mes de verano: {(Q_in*1e-3):.2f} kWh")
print(f"Potencia total necesaria para calefacción en todas las oficinas durante un mes de verano: {(total*1e-3):.2f} kWh")

Potencia necesaria para calefacción en Oficina 1 durante un mes de verano: 59.69 kWh
Potencia necesaria para calefacción en Oficina 2 durante un mes de verano: 90.25 kWh
Potencia necesaria para calefacción en Oficina 3 durante un mes de verano: 67.04 kWh
Potencia necesaria para calefacción en Oficina 4 durante un mes de verano: 71.60 kWh
Potencia necesaria para calefacción en Oficina 5 durante un mes de verano: 71.60 kWh
Potencia necesaria para calefacción en Oficina 6 durante un mes de verano: 90.32 kWh
Potencia necesaria para calefacción en Oficina 7 durante un mes de verano: 90.32 kWh
Potencia necesaria para calefacción en Oficina 8 durante un mes de verano: 113.64 kWh
Potencia necesaria para calefacción en Oficina 9 durante un mes de verano: 124.09 kWh
Potencia necesaria para calefacción en Oficina 10 durante un mes de verano: 90.26 kWh
Potencia necesaria para calefacción en Oficina 11 durante un mes de verano: 90.26 kWh
Potencia necesaria para calefacción en Oficina 12 durante un 